In [ ]:

import cv2
import numpy as np
import math
import pandas as pd
import joblib
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from filterpy.kalman import KalmanFilter
import time
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import pyttsx3
import webbrowser

# === Configuration ===
BASE_DIR = Path.home() / "OneDrive" / "Desktop"
MODEL_PATH_PROTO = BASE_DIR / "pose_deploy_linevec.prototxt"
MODEL_PATH_WEIGHTS = BASE_DIR / "pose_iter_440000.caffemodel"
DATA_PATH = BASE_DIR / "full_pose_data.csv"
EXERCISE_DATA_PATH = BASE_DIR / "exercise_data.pkl"
SITTING_DATA_PATH = BASE_DIR / "sitting_pose_data.csv"
WALKING_DATA_PATH = BASE_DIR / "walking_pose_data.csv"
HEALTH_LOG_PATH = BASE_DIR / "health_log.csv"
REPORT_PATH = BASE_DIR / "data_collection_report.md"
DASHBOARD_PATH = BASE_DIR / "health_dashboard.html"
LOG_PATH = BASE_DIR / "posture_system_log.txt"
POSTURE_MODEL_PATH = BASE_DIR / "posture_model.pkl"
EXERCISE_MODEL_PATH = BASE_DIR / "exercise_model.h5"

inWidth, inHeight = 128, 128
mean_val = (0, 0, 0)
scale = 1.0 / 255.0
threshold = 0.2
MAX_SEQUENCE_LENGTH = 10
VOICE_COOLDOWN = 5  # Seconds between voice alerts
ALERT_THRESHOLD = 60  # Seconds for prolonged incorrect posture alert
EXERCISE_GOAL = 10  # Target reps for progress bar

BODY_PARTS = {
    "Nose": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4,
    "LShoulder": 5, "LElbow": 6, "LWrist": 7, "RHip": 8, "RKnee": 9,
    "RAnkle": 10, "LHip": 11, "LKnee": 12, "LAnkle": 13,
    "REye": 14, "LEye": 15, "REar": 16, "LEar": 17
}

POSE_PAIRS = [
    ["Neck", "RShoulder"], ["RShoulder", "RElbow"], ["RElbow", "RWrist"],
    ["Neck", "LShoulder"], ["LShoulder", "LElbow"], ["LElbow", "LWrist"],
    ["Neck", "RHip"], ["RHip", "RKnee"], ["RKnee", "RAnkle"],
    ["Neck", "LHip"], ["LHip", "LKnee"], ["LKnee", "LAnkle"]
]

# === Logging Setup ===
def setup_logging():
    with open(LOG_PATH, 'a') as f:
        f.write(f"\n=== Session started at {datetime.datetime.now()} ===\n")

def log_message(message):
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open(LOG_PATH, 'a') as f:
        f.write(f"[{timestamp}] {message}\n")
    print(f"[{timestamp}] {message}")

# === Health Log ===
def log_health_data(posture_type, is_correct, duration):
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    data = pd.DataFrame([[timestamp, posture_type, is_correct, duration]], 
                        columns=["Timestamp", "Posture Type", "Is Correct", "Duration (s)"])
    if HEALTH_LOG_PATH.exists():
        data.to_csv(HEALTH_LOG_PATH, mode='a', header=False, index=False)
    else:
        data.to_csv(HEALTH_LOG_PATH, mode='w', header=True, index=False)
    log_message(f"Health data logged: {posture_type}, Correct: {is_correct}, Duration: {duration}s")

# === Voice Engine ===
def init_voice_engine():
    try:
        engine = pyttsx3.init()
        log_message("Voice engine initialized")
        return engine
    except Exception as e:
        log_message(f"Voice engine initialization failed: {e}. Using console output.")
        return None

engine = init_voice_engine()
last_voice_time = 0

def speak(text):
    global last_voice_time
    current_time = time.time()
    if current_time - last_voice_time >= VOICE_COOLDOWN:
        log_message(f"Voice output: {text}")
        if engine:
            try:
                engine.say(text)
                engine.runAndWait()
            except Exception as e:
                log_message(f"Voice error: {e}. Fallback: {text}")
        else:
            print(f"Voice output: {text}")
        last_voice_time = current_time

# === Load Pose Model ===
def load_pose_model():
    try:
        if not (MODEL_PATH_PROTO.exists() and MODEL_PATH_WEIGHTS.exists()):
            raise FileNotFoundError(f"Model files not found at: {MODEL_PATH_PROTO}, {MODEL_PATH_WEIGHTS}")
        net = cv2.dnn.readNetFromCaffe(str(MODEL_PATH_PROTO), str(MODEL_PATH_WEIGHTS))
        log_message("Model loaded successfully")
        return net
    except Exception as e:
        log_message(f"Error loading model: {e}")
        raise

net = load_pose_model()

# === Kalman Filter for Smoothing Keypoints ===
def init_kalman():
    kf = KalmanFilter(dim_x=2, dim_z=2)
    kf.x = np.array([0., 0.])
    kf.F = np.array([[1, 0], [0, 1]])
    kf.H = np.array([[1, 0], [0, 1]])
    kf.P *= 1000.
    kf.R = np.array([[5, 0], [0, 5]])
    return kf

# === Pose Estimator ===
def pose_estimator(frame):
    h, w = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, scale, (inWidth, inHeight), mean_val, swapRB=True, crop=False)
    net.setInput(blob)
    try:
        out = net.forward()
    except Exception as e:
        log_message(f"Error in pose estimation: {e}")
        return frame, [[None] * len(BODY_PARTS)]

    kfs = [init_kalman() for _ in range(len(BODY_PARTS))]
    points_list = []
    num_people = out.shape[0]
    for i in range(num_people):
        points = []
        for j, part in enumerate(BODY_PARTS):
            heatMap = out[i, BODY_PARTS[part], :, :]
            _, conf, _, point = cv2.minMaxLoc(heatMap)
            x = int((w * point[0]) / out.shape[3])
            y = int((h * point[1]) / out.shape[2])
            if conf > threshold:
                kfs[j].predict()
                kfs[j].update(np.array([x, y]))
                points.append(tuple(kfs[j].x.astype(int)))
            else:
                points.append(None)
        points_list.append(points)
        for pair in POSE_PAIRS:
            idFrom, idTo = BODY_PARTS[pair[0]], BODY_PARTS[pair[1]]
            if points[idFrom] and points[idTo]:
                cv2.line(frame, points[idFrom], points[idTo], (0, 255, 0), 2)
                cv2.circle(frame, points[idFrom], 4, (0, 0, 255), -1)
                cv2.circle(frame, points[idTo], 4, (0, 0, 255), -1)
                if points[idFrom]:
                    cv2.putText(frame, pair[0], points[idFrom], cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
                if points[idTo]:
                    cv2.putText(frame, pair[1], points[idTo], cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    return frame, points_list

# === Angle Calculation ===
def calculate_angle(a, b, c):
    if not all([a, b, c]):
        return 0
    try:
        ba = (a[0] - b[0], a[1] - b[1])
        bc = (c[0] - b[0], c[1] - b[1])
        cos_angle = (ba[0]*bc[0] + ba[1]*bc[1]) / (math.hypot(*ba) * math.hypot(*bc) + 1e-6)
        return math.degrees(math.acos(np.clip(cos_angle, -1.0, 1.0)))
    except Exception as e:
        log_message(f"Error calculating angle: {e}")
        return 0

# === Sitting Posture Evaluation ===
def evaluate_sitting_posture(points):
    if not all([points[BODY_PARTS["Neck"]], points[BODY_PARTS["RHip"]], points[BODY_PARTS["LHip"]], 
                points[BODY_PARTS["RKnee"]], points[BODY_PARTS["LKnee"]]]):
        return "Unknown", "Ensure all keypoints are detected", ""

    knee_angle_r = calculate_angle(points[BODY_PARTS["RHip"]], points[BODY_PARTS["RKnee"]], points[BODY_PARTS["RAnkle"]])
    knee_angle_l = calculate_angle(points[BODY_PARTS["LHip"]], points[BODY_PARTS["LKnee"]], points[BODY_PARTS["LAnkle"]])
    back_angle = calculate_angle(points[BODY_PARTS["Neck"]], points[BODY_PARTS["RHip"]], points[BODY_PARTS["RKnee"]])
    
    if knee_angle_r < 30 or knee_angle_l < 30 or points[BODY_PARTS["RHip"]][1] < points[BODY_PARTS["Neck"]][1]:
        return "Standing", "Please sit to evaluate posture", ""

    if 80 <= knee_angle_r <= 110 and 80 <= knee_angle_l <= 110 and 160 <= back_angle <= 180:
        return "Correct", "You are sitting correctly", "Maintain this posture to reduce back strain."
    elif back_angle < 140:
        return "Incorrect", "You are sitting incorrectly, straighten your back", "Sit upright with shoulders back to avoid strain."
    elif abs(points[BODY_PARTS["RHip"]][0] - points[BODY_PARTS["LHip"]][0]) > 50:
        return "Incorrect", "You are sitting incorrectly, align your hips", "Adjust your seat to keep hips level."
    elif knee_angle_r > 130 or knee_angle_l > 130:
        return "Incorrect", "You are sitting incorrectly, adjust knees to ~90 degrees", "Position feet flat on the ground."
    return "Incorrect", "You are sitting incorrectly, maintain proper posture", "Keep back straight and knees bent at 90 degrees."

# === Walking Posture Evaluation ===
def evaluate_walking_posture(points, prev_points):
    if not all([points[BODY_PARTS["Neck"]], points[BODY_PARTS["RHip"]], points[BODY_PARTS["LHip"]], 
                points[BODY_PARTS["RKnee"]], points[BODY_PARTS["LKnee"]], points[BODY_PARTS["RAnkle"]], 
                points[BODY_PARTS["LAnkle"]]]) or not prev_points:
        return "Unknown", "Ensure all keypoints are detected", ""

    back_angle = calculate_angle(points[BODY_PARTS["Neck"]], points[BODY_PARTS["RHip"]], points[BODY_PARTS["RKnee"]])
    r_knee_movement = abs(points[BODY_PARTS["RKnee"]][1] - prev_points[BODY_PARTS["RKnee"]][1]) if prev_points[BODY_PARTS["RKnee"]] else 0
    l_knee_movement = abs(points[BODY_PARTS["LKnee"]][1] - prev_points[BODY_PARTS["LKnee"]][1]) if prev_points[BODY_PARTS["LKnee"]] else 0

    if r_knee_movement < 20 and l_knee_movement < 20:
        return "Stationary", "Please walk to evaluate posture", ""

    if 160 <= back_angle <= 180 and abs(r_knee_movement - l_knee_movement) < 50:
        return "Correct", "You are walking correctly", "Maintain upright posture for better balance."
    elif back_angle < 140:
        return "Incorrect", "You are walking incorrectly, keep your shoulders back", "Stand tall to reduce spinal stress."
    elif abs(r_knee_movement - l_knee_movement) > 100:
        return "Incorrect", "You are walking incorrectly, balance your steps", "Take even strides to avoid strain."
    return "Incorrect", "You are walking incorrectly, maintain upright posture", "Keep head up and shoulders relaxed."

# === Exercise-Specific Feedback ===
def get_exercise_feedback(exercise, points):
    if exercise == "Squat" and points[BODY_PARTS["LKnee"]] and points[BODY_PARTS["RKnee"]]:
        knee_angle = calculate_angle(points[BODY_PARTS["LHip"]], points[BODY_PARTS["LKnee"]], points[BODY_PARTS["LAnkle"]])
        back_angle = calculate_angle(points[BODY_PARTS["Neck"]], points[BODY_PARTS["RHip"]], points[BODY_PARTS["RKnee"]])
        if knee_angle < 70:
            return False, "Your squat is too shallow, go lower", "Lower your hips until thighs are parallel to the ground."
        elif abs(points[BODY_PARTS["LKnee"]][0] - points[BODY_PARTS["RKnee"]][0]) > 50:
            return False, "Your squat is incorrect, keep knees aligned", "Keep knees over toes and avoid leaning."
        elif back_angle < 140:
            return False, "Your squat is incorrect, straighten your back", "Keep your back straight to avoid strain."
        return True, "Your squat is correct", "Great form, keep it up!"
    elif exercise == "Push-up" and points[BODY_PARTS["RShoulder"]] and points[BODY_PARTS["RElbow"]]:
        elbow_angle = calculate_angle(points[BODY_PARTS["RShoulder"]], points[BODY_PARTS["RElbow"]], points[BODY_PARTS["RWrist"]])
        if elbow_angle > 160:
            return False, "Your push-up is incorrect, lower your body more", "Lower until elbows are at 90 degrees."
        return True, "Your push-up is correct", "Maintain this form for maximum benefit."
    elif exercise == "Jumping Jack":
        if points[BODY_PARTS["RWrist"]][1] < points[BODY_PARTS["RShoulder"]][1] - 50:
            return True, "Your jumping jack is correct", "Keep arms and legs synchronized."
        return False, "Your jumping jack is incorrect, raise arms higher", "Raise arms above head during jumps."
    return False, "Maintain proper form", "Follow exercise guidelines for best results."

# === Gesture Recognition ===
def detect_gesture(points):
    if not points or not all([points[BODY_PARTS["RShoulder"]], points[BODY_PARTS["RWrist"]]]):
        return "None"
    shoulder_y = points[BODY_PARTS["RShoulder"]][1]
    wrist_y = points[BODY_PARTS["RWrist"]][1]
    if wrist_y < shoulder_y - 50:
        return "Raise Arm"
    return "None"

# === Repetition Counter ===
class RepCounter:
    def __init__(self, angle_threshold=90, min_angle=30, max_angle=150):
        self.angle_threshold = angle_threshold
        self.min_angle = min_angle
        self.max_angle = max_angle
        self.reps = 0
        self.state = "up"
        self.last_angle = None

    def count_rep(self, angle):
        if angle < self.min_angle or angle > self.max_angle:
            return self.reps, self.state
        if self.last_angle is None:
            self.last_angle = angle
            return self.reps, self.state
        if self.state == "up" and angle < self.angle_threshold:
            self.state = "down"
        elif self.state == "down" and angle > self.angle_threshold:
            self.state = "up"
            self.reps += 1
            speak(f"Rep {self.reps} completed")
        self.last_angle = angle
        return self.reps, self.state

# === Calorie Burn Estimator ===
def estimate_calories(reps, duration, exercise="Squat"):
    met_values = {"Squat": 5.0, "Push-up": 8.0, "Jumping Jack": 8.0}
    met = met_values.get(exercise, 5.0)
    calories = met * 70 * (duration / 3600)
    return round(calories * reps / 10, 2)

# === Health Risk Alert ===
def check_health_risk(posture, duration):
    if posture == "Incorrect" and duration >= ALERT_THRESHOLD:
        speak(f"Warning: Prolonged incorrect posture detected for {int(duration)} seconds. Correct your posture to avoid health risks.")
        log_message(f"Health risk alert: Prolonged incorrect posture ({int(duration)}s)")

# === Progress Bar ===
def draw_progress_bar(frame, reps, goal, y_position):
    progress = min(reps / goal, 1.0)
    bar_width = int(200 * progress)
    cv2.rectangle(frame, (10, y_position), (10 + bar_width, y_position + 20), (0, 255, 0), -1)
    cv2.rectangle(frame, (10, y_position), (210, y_position + 20), (255, 255, 255), 1)
    cv2.putText(frame, f"Progress: {reps}/{goal}", (220, y_position + 15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

# === Health Dashboard ===
def generate_health_dashboard(all_data, exercise_data, sitting_data, walking_data, health_score, total_calories):
    html_content = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <title>Health Dashboard</title>
        <style>
            body {{ font-family: Arial, sans-serif; margin: 20px; }}
            h1 {{ color: #333; }}
            table {{ border-collapse: collapse; width: 100%; }}
            th, td {{ border: 1px solid #ddd; padding: 8px; text-align: left; }}
            th {{ background-color: #f2f2f2; }}
            .highlight {{ color: #0066cc; }}
        </style>
    </head>
    <body>
        <h1>Posture Detection System - Health Dashboard</h1>
        <p><strong>Generated on:</strong> {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}</p>
        <p><strong>Health Score:</strong> <span class="highlight">{health_score:.1f}%</span></p>
        <p><strong>Total Calories Burned:</strong> <span class="highlight">{total_calories:.2f} kcal</span></p>
        <h2>Data Summary</h2>
        <table>
            <tr><th>Category</th><th>Correct</th><th>Incorrect</th></tr>
            <tr><td>Arm Posture</td><td>{sum(1 for _, label in all_data if label == 1)}</td><td>{sum(1 for _, label in all_data if label == 0)}</td></tr>
            <tr><td>Sitting Posture</td><td>{sum(1 for _, label in sitting_data if label == 1)}</td><td>{sum(1 for _, label in sitting_data if label == 0)}</td></tr>
            <tr><td>Walking Posture</td><td>{sum(1 for _, label in walking_data if label == 1)}</td><td>{sum(1 for _, label in walking_data if label == 0)}</td></tr>
            <tr><td>Exercises</td><td colspan="2">
                Push-ups: {sum(1 for _, label in exercise_data if label == 0)}<br>
                Squats: {sum(1 for _, label in exercise_data if label == 1)}<br>
                Jumping Jacks: {sum(1 for _, label in exercise_data if label == 2)}
            </td></tr>
        </table>
        <h2>Health Log</h2>
        <table>
            <tr><th>Timestamp</th><th>Posture Type</th><th>Is Correct</th><th>Duration (s)</th></tr>
    """
    if HEALTH_LOG_PATH.exists():
        df = pd.read_csv(HEALTH_LOG_PATH)
        for _, row in df.iterrows():
            html_content += f"<tr><td>{row['Timestamp']}</td><td>{row['Posture Type']}</td><td>{row['Is Correct']}</td><td>{row['Duration (s)']:.2f}</td></tr>"
    html_content += """
        </table>
    </body>
    </html>
    """
    with open(DASHBOARD_PATH, 'w') as f:
        f.write(html_content)
    log_message(f"Health dashboard generated at {DASHBOARD_PATH}")
    webbrowser.open(DASHBOARD_PATH.as_uri())

# === Data Visualization ===
def plot_data_summary(all_data, sitting_data, walking_data):
    if all_data:
        df = pd.DataFrame(all_data, columns=["elbow_angle", "label"])
        plt.figure(figsize=(8, 6))
        sns.histplot(data=df, x="elbow_angle", hue="label", bins=30)
        plt.title("Elbow Angle Distribution for Arm Postures")
        plt.xlabel("Elbow Angle (degrees)")
        plt.ylabel("Count")
        plot_path = BASE_DIR / "elbow_angle_distribution.png"
        plt.savefig(plot_path)
        plt.close()
        log_message(f"Arm posture plot saved to {plot_path}")
    if sitting_data:
        df = pd.DataFrame(sitting_data, columns=["back_angle", "label"])
        plt.figure(figsize=(8, 6))
        sns.histplot(data=df, x="back_angle", hue="label", bins=30)
        plt.title("Back Angle Distribution for Sitting Postures")
        plt.xlabel("Back Angle (degrees)")
        plt.ylabel("Count")
        plot_path = BASE_DIR / "sitting_angle_distribution.png"
        plt.savefig(plot_path)
        plt.close()
        log_message(f"Sitting posture plot saved to {plot_path}")
    if walking_data:
        df = pd.DataFrame(walking_data, columns=["back_angle", "label"])
        plt.figure(figsize=(8, 6))
        sns.histplot(data=df, x="back_angle", hue="label", bins=30)
        plt.title("Back Angle Distribution for Walking Postures")
        plt.xlabel("Back Angle (degrees)")
        plt.ylabel("Count")
        plot_path = BASE_DIR / "walking_angle_distribution.png"
        plt.savefig(plot_path)
        plt.close()
        log_message(f"Walking posture plot saved to {plot_path}")

# === Generate Report ===
def generate_report(all_data, exercise_data, sitting_data, walking_data, health_score, total_calories):
    report_content = f"""# Posture Detection System Report
Generated on: {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}

## Summary
- **Arm Posture Data Collected**: {len(all_data)} samples
- **Exercise Data Collected**: {len(exercise_data)} sequences
- **Sitting Posture Data Collected**: {len(sitting_data)} samples
- **Walking Posture Data Collected**: {len(walking_data)} samples
- **Health Score**: {health_score:.1f}%
- **Estimated Calories Burned**: {total_calories:.2f} kcal
- **Arm Posture Data File**: {DATA_PATH}
- **Exercise Data File**: {EXERCISE_DATA_PATH}
- **Sitting Data File**: {SITTING_DATA_PATH}
- **Walking Data File**: {WALKING_DATA_PATH}
- **Health Log File**: {HEALTH_LOG_PATH}
- **Dashboard**: {DASHBOARD_PATH}

## Arm Posture Data
- Correct Postures: {sum(1 for _, label in all_data if label == 1)}
- Incorrect Postures: {sum(1 for _, label in all_data if label == 0)}

## Exercise Data
- Push-ups: {sum(1 for _, label in exercise_data if label == 0)}
- Squats: {sum(1 for _, label in exercise_data if label == 1)}
- Jumping Jacks: {sum(1 for _, label in exercise_data if label == 2)}

## Sitting Posture Data
- Correct Sitting: {sum(1 for _, label in sitting_data if label == 1)}
- Incorrect Sitting: {sum(1 for _, label in sitting_data if label == 0)}

## Walking Posture Data
- Correct Walking: {sum(1 for _, label in walking_data if label == 1)}
- Incorrect Walking: {sum(1 for _, label in walking_data if label == 0)}
"""
    with open(REPORT_PATH, 'w') as f:
        f.write(report_content)
    log_message(f"Report generated at {REPORT_PATH}")

# === Mouse Callback for Mode Selection ===
def mouse_callback(event, x, y, flags, param):
    global mode, last_key
    if event == cv2.EVENT_LBUTTONDOWN:
        if 10 <= x <= 110 and 510 <= y <= 530:
            mode = "Squat"
            speak("Entered Squat mode")
            last_key = "Mouse (Squat Mode)"
            log_message("Mouse click: Entered Squat mode")
        elif 120 <= x <= 220 and 510 <= y <= 530:
            mode = "Push-up"
            speak("Entered Push-up mode")
            last_key = "Mouse (Push-up Mode)"
            log_message("Mouse click: Entered Push-up mode")
        elif 230 <= x <= 330 and 510 <= y <= 530:
            mode = "Jumping Jack"
            speak("Entered Jumping Jack mode")
            last_key = "Mouse (Jumping Jack Mode)"
            log_message("Mouse click: Entered Jumping Jack mode")

# === Data Collection ===
def collect_data():
    global mode, last_key
    mode = "None"
    last_key = "None"
    cap = None
    for index in [0, 1, 2, 3]:
        for backend in [cv2.CAP_DSHOW, cv2.CAP_ANY, cv2.CAP_MSMF]:
            cap = cv2.VideoCapture(index, backend)
            if cap.isOpened():
                log_message(f"Webcam opened with index {index} and backend {backend}")
                break
        if cap.isOpened():
            break
    else:
        log_message("No webcam found. Check connection, drivers, or permissions.")
        return

    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
    cv2.namedWindow("Data Collection")
    cv2.setMouseCallback("Data Collection", mouse_callback)
    all_data = []
    exercise_data = []
    sitting_data = []
    walking_data = []
    sequence = []
    exercise_counts = {"Push-up": 0, "Squat": 0, "Jumping Jack": 0}
    sitting_counts = {"Correct": 0, "Incorrect": 0}
    walking_counts = {"Correct": 0, "Incorrect": 0}
    frame_count = 0
    start_time = time.time()
    posture_start_time = time.time()
    last_posture = None
    health_score = 0
    correct_duration = 0
    total_duration = 0
    total_calories = 0
    prev_points = None
    rep_counter = RepCounter()

    log_message("Starting data collection loop")
    print("🔄 Starting data collection loop...")
    print("Press 'g'/'G' (Correct arm posture), 'b'/'B' (Incorrect arm posture),")
    print("'p'/'P' (Push-up mode), 's'/'S' (Squat mode), 'j'/'J' (Jumping Jack mode),")
    print("'m'/'M' (Correct sitting), 'n'/'N' (Incorrect sitting),")
    print("'w'/'W' (Correct walking), 'x'/'X' (Incorrect walking), 't' (Test Squat), 'q' to quit.")
    print("Or click buttons: [Squat] [Push-up] [Jumping Jack] at bottom.")

    while True:
        ret, frame = cap.read()
        if not ret:
            log_message("Failed to capture frame. Retrying...")
            time.sleep(0.1)
            continue
        frame_count += 1
        elapsed_time = time.time() - start_time
        fps = frame_count / elapsed_time if elapsed_time > 0 else 0

        frame, points_list = pose_estimator(frame)
        points = points_list[0] if points_list else [None] * len(BODY_PARTS)
        
        # Evaluate postures
        sitting_label, sitting_feedback, sitting_tip = evaluate_sitting_posture(points)
        walking_label, walking_feedback, walking_tip = evaluate_walking_posture(points, prev_points)
        current_posture = sitting_label if sitting_label != "Standing" else walking_label
        
        # Health risk alert
        current_time = time.time()
        if last_posture and last_posture != current_posture:
            duration = current_time - posture_start_time
            if last_posture == "Correct":
                correct_duration += duration
            total_duration += duration
            log_health_data(last_posture, last_posture == "Correct", duration)
            check_health_risk(last_posture, duration)
        posture_start_time = current_time
        last_posture = current_posture
        health_score = (correct_duration / total_duration * 100) if total_duration > 0 else 0

        # Process sequence for exercises
        angle = calculate_angle(points[BODY_PARTS["RShoulder"]], points[BODY_PARTS["RElbow"]], points[BODY_PARTS["RWrist"]]) if all([points[BODY_PARTS["RShoulder"]], points[BODY_PARTS["RElbow"]], points[BODY_PARTS["RWrist"]]]) else 0
        keypoints = [pt if pt else (0, 0) for pt in points]
        sequence.append(keypoints)
        log_message(f"Sequence length: {len(sequence)}")
        if len(sequence) > MAX_SEQUENCE_LENGTH:
            sequence.pop(0)

        # Evaluate exercise in mode
        exercise_feedback = ""
        exercise_tip = ""
        is_correct_exercise = False
        if mode != "None" and points_list:
            is_correct_exercise, exercise_feedback, exercise_tip = get_exercise_feedback(mode, points)
            if is_correct_exercise:
                reps, _ = rep_counter.count_rep(calculate_angle(points[BODY_PARTS["LHip"]], points[BODY_PARTS["LKnee"]], points[BODY_PARTS["LAnkle"]]) if mode == "Squat" else angle)
                if reps > exercise_counts[mode]:
                    exercise_counts[mode] = reps
                    calories = estimate_calories(reps, elapsed_time, mode)
                    total_calories += calories
                    if len(sequence) >= MAX_SEQUENCE_LENGTH:
                        exercise_data.append([sequence[-MAX_SEQUENCE_LENGTH:], {"Push-up": 0, "Squat": 1, "Jumping Jack": 2}[mode]])
                        log_message(f"Exercise data appended: {len(exercise_data)} entries ({mode})")
                    speak(exercise_feedback)

        # Display information
        color = (0, 255, 0) if current_posture == "Correct" else (0, 0, 255) if current_posture == "Incorrect" else (255, 255, 0)
        if not points_list:
            cv2.putText(frame, "No person detected", (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            cv2.putText(frame, f"Last Key: {last_key}", (10, 130), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
            cv2.putText(frame, f"Mode: {mode}", (10, 160), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
            cv2.putText(frame, f"FPS: {fps:.1f}", (10, 190), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
            cv2.imshow("Data Collection", frame)
            prev_points = points
            key = cv2.waitKey(500) & 0xFF
            if key == 255:
                log_message("No key pressed (ASCII: 255)")
            else:
                last_key = chr(key) if 32 <= key <= 126 else f"Unknown (ASCII: {key})"
                log_message(f"Key pressed: {last_key} (ASCII: {key})")
            if key == ord('q'):
                break
            continue

        cv2.putText(frame, f"Angle: {angle:.1f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
        cv2.putText(frame, f"Sequence: {len(sequence)}/{MAX_SEQUENCE_LENGTH}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
        cv2.putText(frame, f"Last Key: {last_key}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
        cv2.putText(frame, f"Mode: {mode}", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
        cv2.putText(frame, f"Exercises: P:{exercise_counts['Push-up']}, S:{exercise_counts['Squat']}, J:{exercise_counts['Jumping Jack']}", 
                    (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
        cv2.putText(frame, f"Sitting: C:{sitting_counts['Correct']}, I:{sitting_counts['Incorrect']}", 
                    (10, 180), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
        cv2.putText(frame, f"Walking: C:{walking_counts['Correct']}, I:{walking_counts['Incorrect']}", 
                    (10, 210), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
        cv2.putText(frame, f"Sitting: {sitting_label}", (10, 240), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
        cv2.putText(frame, f"Sitting Feedback: {sitting_feedback}", (10, 270), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
        cv2.putText(frame, f"Sitting Tip: {sitting_tip}", (10, 300), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
        cv2.putText(frame, f"Walking: {walking_label}", (10, 330), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
        cv2.putText(frame, f"Walking Feedback: {walking_feedback}", (10, 360), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
        cv2.putText(frame, f"Walking Tip: {walking_tip}", (10, 390), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
        cv2.putText(frame, f"Exercise Feedback: {exercise_feedback}", (10, 420), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
        cv2.putText(frame, f"Exercise Tip: {exercise_tip}", (10, 450), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
        cv2.putText(frame, f"Health Score: {health_score:.1f}%", (10, 480), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
        cv2.putText(frame, f"Calories: {total_calories:.2f} kcal", (10, 510), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
        cv2.putText(frame, f"FPS: {fps:.1f}", (10, 540), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
        if mode in exercise_counts:  # Fix: Only draw progress bar if mode is a valid exercise
            draw_progress_bar(frame, exercise_counts[mode], EXERCISE_GOAL, 570)
        cv2.rectangle(frame, (10, 510), (110, 530), (255, 255, 0), -1)
        cv2.putText(frame, "Squat", (30, 525), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
        cv2.rectangle(frame, (120, 510), (220, 530), (255, 255, 0), -1)
        cv2.putText(frame, "Push-up", (140, 525), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
        cv2.rectangle(frame, (230, 510), (330, 530), (255, 255, 0), -1)
        cv2.putText(frame, "J. Jack", (250, 525), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
        cv2.imshow("Data Collection", frame)

        key = cv2.waitKey(500) & 0xFF
        if key == 255:
            log_message("No key pressed (ASCII: 255)")
        else:
            last_key = chr(key) if 32 <= key <= 126 else f"Unknown (ASCII: {key})"
            log_message(f"Key pressed: {last_key} (ASCII: {key})")
        if key in [ord('g'), ord('G')]:
            all_data.append([angle, 1])
            speak("Correct arm posture saved")
            log_message(f"Arm posture data appended: {len(all_data)} entries")
            last_key = "g/G (Correct Arm)"
        elif key in [ord('b'), ord('B')]:
            all_data.append([angle, 0])
            speak("Incorrect arm posture saved")
            log_message(f"Arm posture data appended: {len(all_data)} entries")
            last_key = "b/B (Incorrect Arm)"
        elif key in [ord('p'), ord('P')]:
            mode = "Push-up"
            speak("Entered Push-up mode")
            last_key = "p/P (Push-up Mode)"
            rep_counter = RepCounter()
        elif key in [ord('s'), ord('S')]:
            mode = "Squat"
            speak("Entered Squat mode")
            last_key = "s/S (Squat Mode)"
            rep_counter = RepCounter()
        elif key in [ord('j'), ord('J')]:
            mode = "Jumping Jack"
            speak("Entered Jumping Jack mode")
            last_key = "j/J (Jumping Jack Mode)"
            rep_counter = RepCounter()
        elif key in [ord('m'), ord('M')]:
            back_angle = calculate_angle(points[BODY_PARTS["Neck"]], points[BODY_PARTS["RHip"]], points[BODY_PARTS["RKnee"]]) if all([points[BODY_PARTS["Neck"]], points[BODY_PARTS["RHip"]], points[BODY_PARTS["RKnee"]]]) else 0
            sitting_data.append([back_angle, 1])
            sitting_counts["Correct"] += 1
            speak("Correct sitting posture saved")
            log_message(f"Sitting posture data appended: {len(sitting_data)} entries (Correct)")
            last_key = "m/M (Correct Sitting)"
        elif key in [ord('n'), ord('N')]:
            back_angle = calculate_angle(points[BODY_PARTS["Neck"]], points[BODY_PARTS["RHip"]], points[BODY_PARTS["RKnee"]]) if all([points[BODY_PARTS["Neck"]], points[BODY_PARTS["RHip"]], points[BODY_PARTS["RKnee"]]]) else 0
            sitting_data.append([back_angle, 0])
            sitting_counts["Incorrect"] += 1
            speak("Incorrect sitting posture saved")
            log_message(f"Sitting posture data appended: {len(sitting_data)} entries (Incorrect)")
            last_key = "n/N (Incorrect Sitting)"
        elif key in [ord('w'), ord('W')]:
            back_angle = calculate_angle(points[BODY_PARTS["Neck"]], points[BODY_PARTS["RHip"]], points[BODY_PARTS["RKnee"]]) if all([points[BODY_PARTS["Neck"]], points[BODY_PARTS["RHip"]], points[BODY_PARTS["RKnee"]]]) else 0
            walking_data.append([back_angle, 1])
            walking_counts["Correct"] += 1
            speak("Correct walking posture saved")
            log_message(f"Walking posture data appended: {len(walking_data)} entries (Correct)")
            last_key = "w/W (Correct Walking)"
        elif key in [ord('x'), ord('X')]:
            back_angle = calculate_angle(points[BODY_PARTS["Neck"]], points[BODY_PARTS["RHip"]], points[BODY_PARTS["RKnee"]]) if all([points[BODY_PARTS["Neck"]], points[BODY_PARTS["RHip"]], points[BODY_PARTS["RKnee"]]]) else 0
            walking_data.append([back_angle, 0])
            walking_counts["Incorrect"] += 1
            speak("Incorrect walking posture saved")
            log_message(f"Walking posture data appended: {len(walking_data)} entries (Incorrect)")
            last_key = "x/X (Incorrect Walking)"
        elif key == ord('t'):
            if len(sequence) >= MAX_SEQUENCE_LENGTH:
                exercise_data.append([sequence[-MAX_SEQUENCE_LENGTH:], 1])
                exercise_counts["Squat"] += 1
                calories = estimate_calories(exercise_counts["Squat"], elapsed_time, "Squat")
                total_calories += calories
                speak("Test Squat sequence saved")
                log_message(f"Exercise data appended: {len(exercise_data)} entries (Test Squat)")
                last_key = "t (Test Squat)"
            else:
                log_message(f"Sequence too short: {len(sequence)}/{MAX_SEQUENCE_LENGTH}")
                last_key = "t (Too Short)"
        elif key == ord('q'):
            break
        prev_points = points

    # Log final health data
    if last_posture:
        duration = time.time() - posture_start_time
        if last_posture == "Correct":
            correct_duration += duration
        total_duration += duration
        log_health_data(last_posture, last_posture == "Correct", duration)
        check_health_risk(last_posture, duration)

    cap.release()
    cv2.destroyAllWindows()
    if all_data:
        pd.DataFrame(all_data, columns=["elbow_angle", "label"]).to_csv(DATA_PATH, index=False)
        log_message(f"Saved {len(all_data)} arm posture entries to {DATA_PATH}")
    if exercise_data:
        pd.DataFrame(exercise_data, columns=["sequence", "exercise_label"]).to_pickle(EXERCISE_DATA_PATH)
        log_message(f"Saved {len(exercise_data)} exercise entries to {EXERCISE_DATA_PATH}")
    if sitting_data:
        pd.DataFrame(sitting_data, columns=["back_angle", "label"]).to_csv(SITTING_DATA_PATH, index=False)
        log_message(f"Saved {len(sitting_data)} sitting posture entries to {SITTING_DATA_PATH}")
    if walking_data:
        pd.DataFrame(walking_data, columns=["back_angle", "label"]).to_csv(WALKING_DATA_PATH, index=False)
        log_message(f"Saved {len(walking_data)} walking posture entries to {WALKING_DATA_PATH}")
    plot_data_summary(all_data, sitting_data, walking_data)
    generate_report(all_data, exercise_data, sitting_data, walking_data, health_score, total_calories)
    generate_health_dashboard(all_data, exercise_data, sitting_data, walking_data, health_score, total_calories)

# === Train Posture Model ===
def train_posture_model():
    if not DATA_PATH.exists():
        log_message("Data file not found.")
        return
    df = pd.read_csv(DATA_PATH)
    if len(df) < 10:
        log_message("Insufficient data for training (need at least 10 samples).")
        return
    if len(df["label"].unique()) < 2:
        log_message("Need both Correct and Incorrect labels for training.")
        return

    X = df[["elbow_angle"]]
    y = df["label"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(X_train, y_train)

    log_message("Posture Classification Report:")
    print("📊 Posture Classification Report:")
    print(classification_report(y_test, clf.predict(X_test)))
    joblib.dump(clf, POSTURE_MODEL_PATH)
    log_message(f"Posture model saved to {POSTURE_MODEL_PATH}")

# === Train Exercise Model (LSTM) ===
def train_exercise_model():
    if not EXERCISE_DATA_PATH.exists():
        log_message("Exercise data file not found.")
        return
    df = pd.read_pickle(EXERCISE_DATA_PATH)
    if len(df) < 10:
        log_message("Insufficient exercise data for training.")
        return

    X = np.array([np.array(seq).flatten() for seq in df["sequence"]], dtype=np.float32)
    y = df["exercise_label"].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = Sequential([
        LSTM(64, input_shape=(MAX_SEQUENCE_LENGTH, len(BODY_PARTS) * 2), return_sequences=False),
        Dense(32, activation='relu'),
        Dense(3, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), verbose=1)
    model.save(EXERCISE_MODEL_PATH)
    log_message(f"Exercise model saved to {EXERCISE_MODEL_PATH}")

# === Live Prediction ===
def live_prediction():
    global mode, last_key
    mode = "None"
    last_key = "None"
    if not POSTURE_MODEL_PATH.exists():
        log_message("Posture model not found.")
        return
    clf = joblib.load(POSTURE_MODEL_PATH)
    exercise_model = tf.keras.models.load_model(EXERCISE_MODEL_PATH) if EXERCISE_MODEL_PATH.exists() else None

    cap = None
    for index in [0, 1, 2, 3]:
        for backend in [cv2.CAP_DSHOW, cv2.CAP_ANY, cv2.CAP_MSMF]:
            cap = cv2.VideoCapture(index, backend)
            if cap.isOpened():
                log_message(f"Webcam opened with index {index} and backend {backend}")
                break
        if cap.isOpened():
            break
    else:
        log_message("No webcam found. Check connection, drivers, or permissions.")
        return

    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
    cv2.namedWindow("Live Prediction")
    cv2.setMouseCallback("Live Prediction", mouse_callback)
    rep_counter = RepCounter()
    sequence = []
    frame_count = 0
    start_time = time.time()
    posture_start_time = time.time()
    last_posture = None
    health_score = 0
    correct_duration = 0
    total_duration = 0
    total_calories = 0
    prev_points = None
    exercise_counts = {"Push-up": 0, "Squat": 0, "Jumping Jack": 0}

    log_message("Starting live prediction")
    print("✅ Press 'p'/'P' (Push-up mode), 's'/'S' (Squat mode), 'j'/'J' (Jumping Jack mode), 'q' to quit.")
    print("Or click buttons: [Squat] [Push-up] [Jumping Jack] at bottom.")
    while True:
        ret, frame = cap.read()
        if not ret:
            log_message("Failed to capture frame.")
            time.sleep(0.1)
            continue
        frame_count += 1
        elapsed_time = time.time() - start_time
        fps = frame_count / elapsed_time if elapsed_time > 0 else 0

        frame, points_list = pose_estimator(frame)
        points = points_list[0] if points_list else [None] * len(BODY_PARTS)
        
        # Evaluate postures
        sitting_label, sitting_feedback, sitting_tip = evaluate_sitting_posture(points)
        walking_label, walking_feedback, walking_tip = evaluate_walking_posture(points, prev_points)
        current_posture = sitting_label if sitting_label != "Standing" else walking_label
        color = (0, 255, 0) if current_posture == "Correct" else (0, 0, 255) if current_posture == "Incorrect" else (255, 255, 0)
        
        # Update health score
        current_time = time.time()
        if last_posture and last_posture != current_posture:
            duration = current_time - posture_start_time
            if last_posture == "Correct":
                correct_duration += duration
            total_duration += duration
            log_health_data(last_posture, last_posture == "Correct", duration)
            check_health_risk(last_posture, duration)
        posture_start_time = current_time
        last_posture = current_posture
        health_score = (correct_duration / total_duration * 100) if total_duration > 0 else 0

        # Process exercise
        angle = calculate_angle(points[BODY_PARTS["RShoulder"]], points[BODY_PARTS["RElbow"]], points[BODY_PARTS["RWrist"]]) if all([points[BODY_PARTS["RShoulder"]], points[BODY_PARTS["RElbow"]], points[BODY_PARTS["RWrist"]]]) else 0
        keypoints = [pt if pt else (0, 0) for pt in points]
        sequence.append(keypoints)
        if len(sequence) > MAX_SEQUENCE_LENGTH:
            sequence.pop(0)

        # Evaluate exercise in mode
        exercise_feedback = ""
        exercise_tip = ""
        is_correct_exercise = False
        if mode != "None" and points_list:
            is_correct_exercise, exercise_feedback, exercise_tip = get_exercise_feedback(mode, points)
            if is_correct_exercise:
                reps, _ = rep_counter.count_rep(calculate_angle(points[BODY_PARTS["LHip"]], points[BODY_PARTS["LKnee"]], points[BODY_PARTS["LAnkle"]]) if mode == "Squat" else angle)
                exercise_counts[mode] = reps
                calories = estimate_calories(reps, elapsed_time, mode)
                total_calories += calories
                speak(exercise_feedback)

        # Display information
        pred = clf.predict([[angle]])[0] if points_list and all([points[BODY_PARTS["RShoulder"]], points[BODY_PARTS["RElbow"]], points[BODY_PARTS["RWrist"]]]) else 0
        label = "Correct" if pred == 1 else "Incorrect"
        color = (0, 255, 0) if pred == 1 else (0, 0, 255)
        feedback = "Straighten your arm" if pred == 0 else "Good arm posture"

        exercise = "Unknown"
        if exercise_model and len(sequence) == MAX_SEQUENCE_LENGTH:
            seq_array = np.array([np.array(sequence).flatten()], dtype=np.float32)
            exercise_pred = exercise_model.predict(seq_array, verbose=0)
            exercise = ["Push-up", "Squat", "Jumping Jack"][np.argmax(exercise_pred)]
        gesture = detect_gesture(points)

        cv2.putText(frame, f"Arm Posture: {label}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
        cv2.putText(frame, f"Arm Feedback: {feedback}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
        cv2.putText(frame, f"Mode: {mode}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
        cv2.putText(frame, f"Exercises: P:{exercise_counts['Push-up']}, S:{exercise_counts['Squat']}, J:{exercise_counts['Jumping Jack']}", 
                    (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
        cv2.putText(frame, f"Sitting: {sitting_label}", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
        cv2.putText(frame, f"Sitting Feedback: {sitting_feedback}", (10, 180), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
        cv2.putText(frame, f"Sitting Tip: {sitting_tip}", (10, 210), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
        cv2.putText(frame, f"Walking: {walking_label}", (10, 240), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
        cv2.putText(frame, f"Walking Feedback: {walking_feedback}", (10, 270), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
        cv2.putText(frame, f"Walking Tip: {walking_tip}", (10, 300), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
        cv2.putText(frame, f"Exercise: {exercise}", (10, 330), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
        cv2.putText(frame, f"Exercise Feedback: {exercise_feedback}", (10, 360), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
        cv2.putText(frame, f"Exercise Tip: {exercise_tip}", (10, 390), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
        cv2.putText(frame, f"Gesture: {gesture}", (10, 420), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
        cv2.putText(frame, f"Health Score: {health_score:.1f}%", (10, 450), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
        cv2.putText(frame, f"Calories: {total_calories:.2f} kcal", (10, 480), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
        cv2.putText(frame, f"FPS: {fps:.1f}", (10, 510), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
        if mode in exercise_counts:  # Fix: Only draw progress bar if mode is a valid exercise
            draw_progress_bar(frame, exercise_counts[mode], EXERCISE_GOAL, 540)
        cv2.rectangle(frame, (10, 510), (110, 530), (255, 255, 0), -1)
        cv2.putText(frame, "Squat", (30, 525), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
        cv2.rectangle(frame, (120, 510), (220, 530), (255, 255, 0), -1)
        cv2.putText(frame, "Push-up", (140, 525), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
        cv2.rectangle(frame, (230, 510), (330, 530), (255, 255, 0), -1)
        cv2.putText(frame, "J. Jack", (250, 525), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
        cv2.imshow("Live Prediction", frame)

        key = cv2.waitKey(500) & 0xFF
        if key == 255:
            log_message("No key pressed (ASCII: 255)")
        else:
            last_key = chr(key) if 32 <= key <= 126 else f"Unknown (ASCII: {key})"
            log_message(f"Key pressed: {last_key} (ASCII: {key})")
        if key in [ord('p'), ord('P')]:
            mode = "Push-up"
            speak("Entered Push-up mode")
            rep_counter = RepCounter()
        elif key in [ord('s'), ord('S')]:
            mode = "Squat"
            speak("Entered Squat mode")
            rep_counter = RepCounter()
        elif key in [ord('j'), ord('J')]:
            mode = "Jumping Jack"
            speak("Entered Jumping Jack mode")
            rep_counter = RepCounter()
        elif key == ord('q'):
            break
        prev_points = points

    # Log final health data
    if last_posture:
        duration = time.time() - posture_start_time
        if last_posture == "Correct":
            correct_duration += duration
        total_duration += duration
        log_health_data(last_posture, last_posture == "Correct", duration)
        check_health_risk(last_posture, duration)

    cap.release()
    cv2.destroyAllWindows()

# === Main Menu ===
if __name__ == "__main__":
    setup_logging()
    print("\n📌 Posture Detection System")
    print("1 - Collect Data")
    print("2 - Train Posture Model")
    print("3 - Train Exercise Model (LSTM)")
    print("4 - Live Prediction")
    choice = input("Enter choice (1-4): ").strip()
    log_message(f"User selected option: {choice}")
    if choice == '1':
        collect_data()
    elif choice == '2':
        train_posture_model()
    elif choice == '3':
        train_exercise_model()
    elif choice == '4':
        live_prediction()
    else:
        log_message("Invalid choice entered.")
        print("❌ Invalid choice.")
